# Notebook summary

This notebook contains three sections:

1. **Preprocesing:** removing data dependent attributes as we already known that are not much correlated with the response
2. **Tuning max_depth for regression trees:** tuning this hyperparameter over the whole principal components
3. **Tuning number of principal components:** trying to see if we can reduce the dimensionality of the problem
4. **Checking performance of final model:** comparing final model obtained against the one built before

In [2]:
from pyspark.ml.feature import PCA
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml import Pipeline
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
import matplotlib.pyplot as plt
from pyspark.ml.feature import VectorSlicer
from pyspark.ml.linalg import Vectors
from pyspark.sql.types import Row

In [3]:
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
import numpy as np
wind = spark.table("wind")

#Filtering the dataset into train/validation/test
wind_train=wind.filter(wind["year"]<=2006)
wind_validation=wind[(wind.year==2007)|(wind.year==2008)]
wind_test=wind.filter(wind["year"]>=2009)
wind_train_validation=wind.filter(wind["year"]<2009)


#Function to preproces the data
def preprocessing(data):
  notignore=["u100","v100","u10","v10","ienss","iews"]
  col_selected = []
  for idx_1,name in enumerate(notignore):
    for idx_2,name in enumerate(data.columns):
      if notignore[idx_1] in name:
        list=col_selected.append(name)
  assembler = VectorAssembler(inputCols=col_selected,outputCol="features")
  temp = assembler.transform(data).select(['energy','features'])
  return (temp)

#Applying the function to preprocessing
wind_train=preprocessing(wind_train)
wind_validation=preprocessing(wind_validation)
wind_test=preprocessing(wind_test)
wind_train_validation=preprocessing(wind_train_validation)

wind_train_validation.cache()
wind_test.cache()

#Obtaining Principal Components
pca = PCA(k=225,inputCol="features", outputCol="pcaFeatures")
model_pca = pca.fit(wind_train)

wind_train = model_pca.transform(wind_train)
wind_validation = model_pca.transform(wind_validation)

wind_train.cache()
wind_validation.cache()

Out[ 28 ]: DataFrame[energy: double, features: vector, pcaFeatures: vector]

+-------+--------------------+--------------------+
 energy| features| pcaFeatures|
+-------+--------------------+--------------------+
 402.71|[-2.02538646989,-...|[34.7957985497175...|
 696.8|[-4.05826760046,-...|[37.3670339791732...|
1591.15|[-5.57638932997,-...|[35.4122585897114...|
1338.62|[-4.92247748921,-...|[18.9984014919693...|
 562.5|[-3.74020488111,-...|[7.58860370420176...|
 232.3|[0.475695538666,0...|[-27.741355922763...|
 329.95|[4.53308772824,4....|[-46.103577954492...|
 960.51|[3.77036495717,3....|[-43.402836871874...|
 194.62|[3.80907653815,3....|[-43.354363252422...|
 358.51|[3.53338730608,3....|[-40.726031764995...|
 808.8|[3.31838109284,3....|[-45.978620088557...|
 93.36|[3.84098743597,3....|[-50.399112540047...|
 155.94|[4.57284556816,4....|[-44.539642600604...|
 0.01|[2.41284397575,2....|[-36.562329285412...|
 4.85|[-0.151013569522,...|[-20.740270169439...|
 218.76|[1.09351144582,1....|[-18.305213678735...|
 906.21|[0.973191667119,0...|[-22.431661712009...|
 201.42|[1.72335933084,1....|[-43.869591210133...|
 641.34|[3.9607840852,3.7...|[-58.876725662621...|
1524.05|[4.87678379175,4....|[-69.517371298302...|
+-------+--------------------+--------------------+
only showing top 20 rows

+--------------------+--------------------+
 pcaFeatures| featuressort|
+--------------------+--------------------+
[34.7957985497175...|[-15.621630968522...|
[37.3670339791732...|[2.2590167486137984]|
[35.4122585897114...|[20.842330341741913]|
[18.9984014919693...| [37.94060197515303]|
[7.58860370420176...| [30.32367591118965]|
[-27.741355922763...|[27.983006840994562]|
[-46.103577954492...|[3.9954795885050407]|
[-43.402836871874...| [1.065336208496164]|
[-43.354363252422...| [8.390212291374205]|
[-40.726031764995...| [9.37210561353823]|
[-45.978620088557...| [16.71282502492763]|
[-50.399112540047...|[12.249729191881638]|
[-44.539642600604...|[0.4950952454664227]|
[-36.562329285412...|[17.420484694983863]|
[-20.740270169439...| [26.60712765717549]|
[-18.305213678735...| [14.25349832047063]|
[-22.431661712009...|[18.334449818290754]|
[-43.869591210133...|[30.369513108706798]|
[-58.876725662621...|[24.847353493245315]|
[-69.517371298302...| [24.8745097956195]|
+--------------------+--------------------+
only showing top 20 rows

In [5]:
##In this case, as we are tunning just the max depth of the decision tree regresor, we are using the same PC on each iteration - so it is convenient
##to cache that transformation (previously done)

d={}
for i in range(1,10):
  dt = DecisionTreeRegressor(featuresCol="pcaFeatures",labelCol="energy",maxDepth=i)
  pipeline_dt = Pipeline(stages=[dt])
  model_dt= pipeline_dt.fit(wind_train)

  predictions = model_dt.transform(wind_validation)

  evaluator = RegressionEvaluator(labelCol="energy", predictionCol="prediction", metricName="mae")
  mae_train = evaluator.evaluate(predictions)
  print("MAE with all principal components and maxdepth={}: {}".format(i,mae_train))
  d[i]=mae_train

plt.plot(d.keys(),d.values())
plt.show()

MAE with all principal components and maxdepth=1: 436.369304139
MAE with all principal components and maxdepth=2: 414.804000712
MAE with all principal components and maxdepth=3: 379.359062634
MAE with all principal components and maxdepth=4: 352.304887965
MAE with all principal components and maxdepth=5: 338.541822319
MAE with all principal components and maxdepth=6: 333.411654299
MAE with all principal components and maxdepth=7: 340.658323997
MAE with all principal components and maxdepth=8: 349.940077781
MAE with all principal components and maxdepth=9: 369.604472469

In [6]:
#From this result we can see that best max depth is equal to 5
#Let´s check with the test set

In [7]:
# 2 - Evaluating the model after setting best max_depth = 5
##Obtaining PCA with train+validation

pca = PCA(k=225,inputCol="features", outputCol="pcaFeatures")
model_pca_train_validation = pca.fit(wind_train_validation)

#Transforming the data with the fitted pc
wind_train_validation_pca=model_pca_train_validation.transform(wind_train_validation)
wind_test_pca = model_pca_train_validation.transform(wind_test)

#Fitting a decision tree with the train+validation with pca already obtained
dt = DecisionTreeRegressor(featuresCol="pcaFeatures",labelCol="energy",maxDepth=5)
pipeline_dt = Pipeline(stages=[dt])
model_dt= pipeline_dt.fit(wind_train_validation_pca)

#Making predictions over test partition
predictions=model_dt.transform(wind_test_pca)

#Evaluating the model with MAE
evaluator = RegressionEvaluator(labelCol="energy", predictionCol="prediction", metricName="mae")
mae_train = evaluator.evaluate(predictions)

print("MAE with all principal components and maxdepth={}: {}".format(5,mae_train))

MAE with all principal components and maxdepth=5: 351.087104377

In [8]:
#Apartado 3 1D search for #pca in range 10 to 190 with steps of 10

for i in [i*10 for i in range(1,20)]:
  #pca_train = PCA(k=i, inputCol="features")
  
  pca_train=VectorSlicer(inputCol="pcaFeatures", outputCol="selectedpcafeatures", indices=range(1,i))
  
  dt_train = DecisionTreeRegressor(featuresCol=pca_train.getOutputCol(), 
                             labelCol="energy",maxDepth=5)
  
  pipeline_train = Pipeline(stages=[pca_train, dt_train])

  model_train = pipeline_train.fit(wind_train)
  predictions_validation = model_train.transform(wind_validation)

  evaluator = RegressionEvaluator(labelCol="energy", predictionCol="prediction", metricName="mae")
  mae_validation = evaluator.evaluate(predictions_validation)

  print("MAE with {} pca components: {}".format(i,mae_validation))
  

MAE with 10 pca components: 473.166612927
MAE with 20 pca components: 456.020076639
MAE with 30 pca components: 460.311806716
MAE with 40 pca components: 451.238232819
MAE with 50 pca components: 450.018918342
MAE with 60 pca components: 451.642750891
MAE with 70 pca components: 452.933037073
MAE with 80 pca components: 454.279503448
MAE with 90 pca components: 453.757230738
MAE with 100 pca components: 453.757230738
MAE with 110 pca components: 453.757230738
MAE with 120 pca components: 453.757230738
MAE with 130 pca components: 453.757230738
MAE with 140 pca components: 453.757230738
MAE with 150 pca components: 453.757230738
MAE with 160 pca components: 453.757230738
MAE with 170 pca components: 453.757230738
MAE with 180 pca components: 451.233690323
MAE with 190 pca components: 452.849861223
+--------------------+--------------------+
 pcaFeatures| featuressort|
+--------------------+--------------------+
[34.7957985497175...|[-15.621630968522...|
[37.3670339791732...|[2.2590167486137984]|
[35.4122585897114...|[20.842330341741913]|
[18.9984014919693...| [37.94060197515303]|
[7.58860370420176...| [30.32367591118965]|
[-27.741355922763...|[27.983006840994562]|
[-46.103577954492...|[3.9954795885050407]|
[-43.402836871874...| [1.065336208496164]|
[-43.354363252422...| [8.390212291374205]|
[-40.726031764995...| [9.37210561353823]|
[-45.978620088557...| [16.71282502492763]|
[-50.399112540047...|[12.249729191881638]|
[-44.539642600604...|[0.4950952454664227]|
[-36.562329285412...|[17.420484694983863]|
[-20.740270169439...| [26.60712765717549]|
[-18.305213678735...| [14.25349832047063]|
[-22.431661712009...|[18.334449818290754]|
[-43.869591210133...|[30.369513108706798]|
[-58.876725662621...|[24.847353493245315]|
[-69.517371298302...| [24.8745097956195]|
+--------------------+--------------------+
only showing top 20 rows

In [9]:
#From the results on previous part we can assume that more than 20 PCA does not improve the performance model, in fact it reduces the MAE. 
#Let´s check the range from 10 to 20 to check wheter we can reduce the amount of predictors or not

In [10]:
#Apartado 3 1D search for #pca in range 10 to 190 with steps of 10

for i in [i for i in range(10,30)]:
  #pca_train = PCA(k=i, inputCol="features")
  
  pca_train=VectorSlicer(inputCol="pcaFeatures", outputCol="selectedpcafeatures", indices=range(1,i))
  
  dt_train = DecisionTreeRegressor(featuresCol=pca_train.getOutputCol(), 
                             labelCol="energy",maxDepth=5)
  
  pipeline_train = Pipeline(stages=[pca_train, dt_train])

  model_train = pipeline_train.fit(wind_train)
  predictions_validation = model_train.transform(wind_validation)

  evaluator = RegressionEvaluator(labelCol="energy", predictionCol="prediction", metricName="mae")
  mae_validation = evaluator.evaluate(predictions_validation)

  print("MAE with {} pca components: {}".format(i,mae_validation))

MAE with 10 pca components: 473.166612927
MAE with 11 pca components: 467.37234066
MAE with 12 pca components: 467.37234066
MAE with 13 pca components: 466.064541165
MAE with 14 pca components: 466.046994549
MAE with 15 pca components: 466.046994549
MAE with 16 pca components: 463.047576894
MAE with 17 pca components: 462.921110268
MAE with 18 pca components: 462.921110268
MAE with 19 pca components: 455.992074971
MAE with 20 pca components: 456.020076639
MAE with 21 pca components: 456.020076639
MAE with 22 pca components: 454.653051228
MAE with 23 pca components: 455.218783522
MAE with 24 pca components: 455.218783522
MAE with 25 pca components: 455.218783522
MAE with 26 pca components: 455.218783522
MAE with 27 pca components: 455.218783522
MAE with 28 pca components: 460.311806716
MAE with 29 pca components: 460.311806716

In [11]:
#From the results above, we can see that best number of principal components is equal to 19

In [12]:
# Apartado 4
pca_train_validation = PCA(k=19, inputCol="features")
dt_train_validation = DecisionTreeRegressor(featuresCol=pca_train_validation.getOutputCol(), 
                           labelCol="energy",maxDepth=5)

pipeline_train_validation = Pipeline(stages=[pca_train_validation, dt_train_validation])

model_train_validation = pipeline_train_validation.fit(wind_train_validation)
predictions_train_validation = model_train_validation.transform(wind_test)

evaluator = RegressionEvaluator(labelCol="energy", predictionCol="prediction", metricName="mae")
mae_train = evaluator.evaluate(predictions_train_validation)

print("MAE with all components components: {}".format(mae_train))

MAE with all components components: 352.486625927

In [13]:
#From this result, we can observe that a model using the first 4 principal components has almost the same MAE result as using all of them, so in this case the dimensionality of the problem will be reduced a lot, improving the time-performance of the model